# Areas of Interest to Django
Import SpatiaLite records to a GeoDjango database

### Import libraries, define settings, import model

In [1]:
import os
import django
from django.db import transaction
from django.contrib.gis.geos import GEOSGeometry
import sqlite3
from asgiref.sync import sync_to_async

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from whale.models import GEOINTDiscovery as DjangoTable
from whale.models import AreaOfInterest as ForeignDjangoTable

### User defined variables

In [2]:
sdb = "C:/gis/gaia/data/databases/gaia.db"
sdb_table_name = "gegd"
geometry_col = 'geometry'

### Perform async insertion to database

In [3]:
@sync_to_async
def insert_data(**kwargs):
    """ Insert Areas of Interest into the GeoDjango database"""
    if 'geometry_text' in kwargs:
        kwargs['geometry'] = GEOSGeometry(kwargs.pop('geometry_text'))
    if 'aoi_id' in kwargs:
        kwargs['aoi_id'] = ForeignDjangoTable.objects.get(id=kwargs.pop('aoi_id'))
    
    DjangoTable.objects.update_or_create(**kwargs)

conn = sqlite3.connect(sdb)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

pragma_statement = f"PRAGMA table_info({sdb_table_name});"
print(pragma_statement)
cursor = conn.execute(pragma_statement)
columns = [row[1] for row in cursor.fetchall()]
print(columns)

columns_with_geom = [
    f"AsText({geometry_col}) AS geometry_text" if col == geometry_col else col
    for col in columns
]
print(columns_with_geom)
query = f"SELECT {', '.join(columns_with_geom)} FROM {sdb_table_name};"
print(query)
cursor.execute(query)
rows = cursor.fetchall()

conn.close()

import asyncio

async def commit_rows():
    for row in rows:
        data = dict(zip(columns, row))
        await insert_data(**data)

await commit_rows()

PRAGMA table_info(gegd);
['id', 'aoi_id', 'legacy_id', 'factory_order_number', 'acquisition_date', 'source', 'source_unit', 'product_type', 'cloud_cover', 'off_nadir_angle', 'sun_elevation', 'sun_azimuth', 'ground_sample_distance', 'data_layer', 'legacy_description', 'color_band_order', 'asset_name', 'per_pixel_x', 'per_pixel_y', 'crs_from_pixels', 'age_days', 'ingest_date', 'company_name', 'copyright', 'niirs', 'geometry']
['id', 'aoi_id', 'legacy_id', 'factory_order_number', 'acquisition_date', 'source', 'source_unit', 'product_type', 'cloud_cover', 'off_nadir_angle', 'sun_elevation', 'sun_azimuth', 'ground_sample_distance', 'data_layer', 'legacy_description', 'color_band_order', 'asset_name', 'per_pixel_x', 'per_pixel_y', 'crs_from_pixels', 'age_days', 'ingest_date', 'company_name', 'copyright', 'niirs', 'AsText(geometry) AS geometry_text']
SELECT id, aoi_id, legacy_id, factory_order_number, acquisition_date, source, source_unit, product_type, cloud_cover, off_nadir_angle, sun_eleva

C:\Users\jeffrey.wyman\AppData\Local\anaconda3\envs\gaia\Lib\site-packages\django\db\models\fields\__init__.py:1659: RuntimeWarning: DateTimeField GEOINTDiscovery.acquisition_date received a naive datetime (2023-10-01 06:02:38) while time zone support is active.
  warnings.warn(
C:\Users\jeffrey.wyman\AppData\Local\anaconda3\envs\gaia\Lib\site-packages\django\db\models\fields\__init__.py:1659: RuntimeWarning: DateTimeField GEOINTDiscovery.ingest_date received a naive datetime (2023-10-01 12:34:43) while time zone support is active.
  warnings.warn(
C:\Users\jeffrey.wyman\AppData\Local\anaconda3\envs\gaia\Lib\site-packages\django\db\models\fields\__init__.py:1659: RuntimeWarning: DateTimeField GEOINTDiscovery.acquisition_date received a naive datetime (2023-01-28 06:25:12) while time zone support is active.
  warnings.warn(
C:\Users\jeffrey.wyman\AppData\Local\anaconda3\envs\gaia\Lib\site-packages\django\db\models\fields\__init__.py:1659: RuntimeWarning: DateTimeField GEOINTDiscovery.in

# End